In [13]:
from selenium import webdriver
import datetime
import time
from PIL import Image
from io import BytesIO
import tesserocr

def daojia(username,password):

    url = 'https://plogin.m.jd.com/user/login.action?appid=121&returnurl=https%3A%2F%2Fdaojia.jd.com%2Fclient%3FfunctionId%3Dlogin%2Fpassport%26platCode%3DH5%26appName%3Dpaidaojia%26appVersion%3D5.7.0%26body%3D%257B%2522returnLink%2522%253A%2520%2522https%253A%252F%252Fdaojia.jd.com%252Fhtml%252Findex.html%2523user%252FLID%253A2%2522%257D%26signKey%3D1d973ce5ab8bb79c0d2c5d4e973e5c37'
    driver = webdriver.Chrome() #声明浏览器对象
    driver.get(url) # 访问网页
    driver.maximize_window()
    #driver.find_element_by_id('username').send_keys(username) # 提取账号输入框，并输入用户名
    #driver.find_element_by_id('password').send_keys(password) # 提取密码输入框，并输入密码
    
    
    '''
    imgcode = driver.find_element_by_id('imgCode') # 获取验证码图片
    location= imgcode.location
    size = imgcode.size
    print(location)
    print(size)
    left,top,right,bottom = location['x'],location['y'],location['x']+size['width'],location['y']+size['height']
    print(left,top,right,bottom)
    '''
  
    screenshot = driver.get_screenshot_as_png()
    screenshot = Image.open(BytesIO(screenshot))
    screenshot.save('screenshot.png')
    captcha = screenshot.crop((1790,370,1930,460))
    captcha.save('imgcode.png')
    
    driver.quit() # 退出浏览器
    #driver.find_element_by_css_selector('a.logbtn.submit').click() # 点击登陆按钮
    #driver.implicitly_wait(5) # 隐式等待
    
    '''
    
    driver.implicitly_wait(5) # 隐式等待
    link = driver.find_element_by_id('nav_key_up_53').get_attribute('href') # 获取签到链接
    driver.get(link) # 访问签到链接
    driver.implicitly_wait(5)  # 隐式等待
    driver.find_element_by_name('qdxq').click() # 点击高兴表情
    driver.find_element_by_class_name('btn').submit() # 点击签到
    info = driver.find_element_by_css_selector('p.f14.mb10').text #获取签到后信息
    print('时间：{}\n'.format(datetime.datetime.now()),info) #打印签到信息
    driver.quit() # 退出浏览器
    '''
    


username = '' # 你的账号
password = '' # 你的密码
daojia(username,password)



{'x': 895, 'y': 194}
{'height': 32, 'width': 77}
895 194 972 226



In [2]:
import tesserocr
from PIL import Image

image = Image.open('imgcode.png')
image = image.convert('L')

image.show()
#result = tesserocr.image_to_text(image)
#print(result)

##  验证码

字符型图片验证码识别完整过程及Python实现

https://www.cnblogs.com/beer/p/5672678.html


验证码——python去除干扰线

https://blog.csdn.net/eddy_zheng/article/details/48895085


python+机器学习+验证码识别+源码

https://bbs.ichunqiu.com/thread-38774-1-1.html?from=beef

有了Python+Tensorflow！所有验证码识别都能快人一步！快速识别

http://www.sohu.com/a/213300147_100033985


python爬虫验证码识别（去除干扰线）

https://blog.csdn.net/weixin_40267472/article/details/81384624

### 从验证码接口 批量下载验证码图片

京东的验证码接口url单独打开是没有用的，必须先打开京东登陆页面的url
所以需要用session 会话维持

In [18]:
import requests


def captcha(url,imgurl,times):
    s = requests.Session()
    s.get(url)
    headers={"User-Agent": "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36",
             "Connection": "keep-alive",
             "Upgrade-Insecure-Requests":"1",
             "Host":"plogin.m.jd.com"}
    for i in range(times):
        img = s.get(imgurl,headers=headers).content
        with open('Captcha/{}.jpeg'.format(i),'wb') as f:
            print('pic{} downloading...'.format(i))
            f.write(img)    
    print('done')

url = 'https://plogin.m.jd.com/user/login.action?appid=121&returnurl=https%3A%2F%2Fdaojia.jd.com%2Fclient%3FfunctionId%3Dlogin%2Fpassport%26platCode%3DH5%26appName%3Dpaidaojia%26appVersion%3D5.7.0%26body%3D%257B%2522returnLink%2522%253A%2520%2522https%253A%252F%252Fdaojia.jd.com%252Fhtml%252Findex.html%2523user%252FLID%253A2%2522%257D%26signKey%3D1d973ce5ab8bb79c0d2c5d4e973e5c37'
imgurl = 'https://plogin.m.jd.com/cgi-bin/m/authcode?mod=login' #验证码生产接口

if __name__=='__main__':
    
    captcha(url,imgurl,10)

pic0 downloading...
pic1 downloading...
pic2 downloading...
pic3 downloading...
pic4 downloading...
pic5 downloading...
pic6 downloading...
pic7 downloading...
pic8 downloading...
pic9 downloading...
done


## 图片预处理

将RGB彩图转为灰度图

将灰度图按照设定阈值转化为二值图

In [24]:
from PIL import Image
import tesserocr

img = Image.open('imgcode.png') # 读取原始图片
grey_img = img.convert('L')  # 转化为灰度图

#二值化函数
def get_bin_table(threshold):
    """
    获取灰度转二值的映射table
    """
    table = []
    for i in range(256):
        if i < threshold:
            table.append(0)
        else:
            table.append(1)
 
    return table

table = get_bin_table(200)
out = grey_img.point(table,'1')
out.show()


result = tesserocr.image_to_text(out)
print(result)